## TF-IDF 알고리즘
- 예제 : https://wikidocs.net/24603

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import os
print(os.listdir("../input/t-academy-recommendation2/movies/"))

In [ ]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = '../input/t-academy-recommendation2/movies/'

In [ ]:
data = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
data.head(2)

In [ ]:
# overview의 항목 추출 
data.columns

In [ ]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

In [ ]:
data = data.loc[0:20000].reset_index(drop=True)

In [ ]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
# https://wikidocs.net/22530
tfidf = TfidfVectorizer(stop_words='english')

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생하신 분은 TF-IDF의 파라미터를 수정해줘서 다시 돌리면 됩니다. 
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행해보시길 바랍니다. 
# data = data.loc[0:10000].reset_index(drop=True)

In [ ]:
cosine_matrix.shape

In [ ]:
np.round(cosine_matrix, 4)

In [ ]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [ ]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

In [ ]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores